In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import sys
import datetime as datetime
from datetime import datetime as dt
from datetime import timedelta as td
from datetime import date as dater

df = pd.read_csv(r'C:/Users/enoch/Desktop/MLvsCOVID/completedataset.csv')
df=df.set_index(['county','date'])

df

retailandrec  groceryandpharm  parks  transit  work  \
county  date                                                              
Alameda 2020-02-20           1.0              1.0   13.0      2.0   0.0   
        2020-02-21           1.0              1.0    8.0     -1.0   3.0   
        2020-02-26           2.0              2.0   20.0      3.0   2.0   
        2020-02-28           3.0              5.0    8.0      0.0   4.0   
        2020-02-29           2.0              7.0   14.0     -4.0   3.0   
...                          ...              ...    ...      ...   ...   
Yuba    2020-07-13         -13.0              8.0    NaN     -9.0 -32.0   
        2020-07-14          -9.0              8.0    NaN     -8.0 -31.0   
        2020-07-15          -9.0              9.0    NaN    -14.0 -34.0   
        2020-07-16         -12.0             18.0    NaN    -12.0 -33.0   
        2020-07-17         -13.0              9.0    NaN    -17.0 -34.0   

                    residential  confirmed_cases  deaths  new_confirmed_cases  \
county  date                                                                    
Alameda 2020-02-20          0.0                0       0                  0.0   
        2020-02-21          0.0                0       0                  0.0   
        2020-02-26         -1.0                0       0                  0.0   
        2020-02-28          0.0                0       0                  0.0   
        2020-02-29         -1.0                0       0                  0.0   
...                         ...              ...     ...                  ...   
Yuba    2020-07-13          6.0              211       3                 13.0   
        2020-07-14          7.0              221       3                 10.0   
        2020-07-15          7.0              231       3                 10.0   
        2020-07-16          7.0              236       3                  5.0   
        2020-07-17          8.0              252       3                 16.0   

                    new_deaths  
county  date                    
Alameda 2020-02-20         0.0  
        2020-02-21         0.0  
        2020-02-26         0.0  
        2020-02-28         0.0  
        2020-02-29         0.0  
...                        ...  
Yuba    2020-07-13         0.0  
        2020-07-14         0.0  
        2020-07-15         0.0  
        2020-07-16         0.0  
        2020-07-17         0.0  

[7080 rows x 10 columns]

In [3]:
def adddelaycolumn(numdays):
    relevantdate = df.index.tolist() 
    bchanges =[]

    for date in relevantdate:

        newdate = dt.strptime(date[1], '%Y-%m-%d')+td(days=numdays)
        #print (newdate)
        #print(str(newdate)[0:10])
        try:
            bchanges.append(df.loc[date[0]].loc[str(newdate)[0:10]].new_confirmed_cases)
        except:
            bchanges.append(np.nan)
    

    string= str(numdays)+'dayslaternewcases'
    #print(changes) 
    df[string]=bchanges

   


adddelaycolumn(11)
#df.corr()

In [4]:
# Remove rows with missing target, separate target from predictors

needsprediction=df[df['11dayslaternewcases'].isnull()]
#print(needsprediction)
testing=df[df['11dayslaternewcases'].notnull()]

In [5]:
#drop values with no target
df.dropna(subset=['11dayslaternewcases'],inplace=True)
#print(df)


In [6]:
# Break off validation set from training data
X = df.drop(columns=['11dayslaternewcases'])
y=df['11dayslaternewcases']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

#print(X)

In [6]:

# Function for comparing different approaches
   

In [7]:

#Create imputer
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

#imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_train.index = X_train.index
imputed_X_valid.index = X_valid.index

print(imputed_X_valid)

                        retailandrec  groceryandpharm      parks    transit  \
county      date                                                              
San Joaquin 2020-03-11      7.000000         8.000000  34.000000  -2.000000   
Plumas      2020-05-10    -31.083247        -3.459948   2.131226 -28.510194   
Inyo        2020-05-11    -31.083247        -3.459948 -34.000000 -28.510194   
Stanislaus  2020-05-15    -34.000000         1.000000  32.000000 -15.000000   
San Benito  2020-05-24    -14.000000        -1.000000 -12.000000 -28.510194   
...                              ...              ...        ...        ...   
Santa Clara 2020-03-01      1.000000         6.000000  17.000000  -8.000000   
Siskiyou    2020-05-08    -19.000000        23.000000   2.131226  22.000000   
Sacramento  2020-04-15    -40.000000        -9.000000   5.000000 -53.000000   
Solano      2020-02-26      5.000000         3.000000  18.000000   6.000000   
Fresno      2020-06-16    -21.000000         7.00000

In [8]:
print("MAE (Imputation):")
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(imputed_X_train, y_train)
preds = model.predict(imputed_X_valid)
print(mean_absolute_error(y_valid, preds))


MAE (Imputation):
25.089992378727192


In [9]:
print(y_valid)
print(preds)

county      date      
Amador      2020-06-01      0.0
Orange      2020-05-15    109.0
Napa        2020-06-12      6.0
Tulare      2020-06-28    211.0
El Dorado   2020-05-29      2.0
                          ...  
Mono        2020-04-03      0.0
Solano      2020-06-25    264.0
Mono        2020-06-08      0.0
Sacramento  2020-02-26      0.0
Ventura     2020-05-06      0.0
Name: 11dayslaternewcases, Length: 1208, dtype: float64
[  0.14       120.58         7.44       ...   0.12333333   0.98
  13.02      ]


In [10]:
lookhere=needsprediction['11dayslaternewcases']
testing=needsprediction.drop(columns=['11dayslaternewcases'])
testing=pd.DataFrame(imputer.transform(testing))
#testing.columns = needsprediction.columns
testing['predictions']= model.predict(testing)
testing.index = needsprediction.index

print(testing)




                       0     1          2     3     4     5       6      7  \
county  date                                                                 
Alameda 2020-06-30 -37.0 -11.0  37.000000 -59.0 -53.0  20.0  5964.0  135.0   
        2020-07-01 -36.0 -10.0  32.000000 -57.0 -53.0  20.0  6156.0  136.0   
        2020-07-02 -32.0  -1.0  38.000000 -57.0 -55.0  19.0  6384.0  138.0   
        2020-07-03 -37.0   0.0  47.000000 -60.0 -67.0  22.0  6472.0  140.0   
        2020-07-04 -59.0 -19.0  20.000000 -54.0 -32.0  11.0  6556.0  140.0   
...                  ...   ...        ...   ...   ...   ...     ...    ...   
Yuba    2020-07-06  -7.0  13.0  -0.044512   1.0 -36.0   7.0   139.0    2.0   
        2020-07-07  -2.0   9.0  -0.044512  -6.0 -32.0   6.0   147.0    2.0   
        2020-07-08  -4.0   8.0  -0.044512  -9.0 -32.0   7.0   154.0    3.0   
        2020-07-09  -7.0  17.0  -0.044512  -8.0 -33.0   7.0   163.0    3.0   
        2020-07-10  -9.0  11.0  -0.044512  -8.0 -31.0   8.0   18

NameError: name 'df' is not defined